In [1]:
import sys
import os

In [2]:
import torch
import torch.nn as nn

In [3]:
sys.path.append(os.path.abspath("../.."))
from utils import read_features, read_targets, print_info_features, print_info_targets, train_val_split, get_dimensions, scale, \
    metrics_r, get_device, plot_history

In [4]:
sys.path.append(os.path.abspath(".."))
from PotteryDataset import create_pottery_dataloaders, create_pottery_datasets, feature_types, feature_type_combos
from PotteryChronologyPredictor import PotteryChronologyPredictor, train, tune

## Set Working Device

In [5]:
device = get_device()

PyTorch Version: 2.5.1
CUDA is available
GPU: NVIDIA GeForce RTX 4080
Using Device: cuda


## Read Features and Targets

In [6]:
path = os.path.abspath(os.path.join(os.getcwd(), "../../../data/chronology_prediction"))

In [7]:
targets = ["StartYear", "YearRange"]

In [8]:
X = read_features(path, f_type="tensors")
y = read_targets(path, targets, f_type="np")

Loaded X_train_tfidf
Loaded X_train_bert
Loaded X_train_cannyhog
Loaded X_train_resnet
Loaded X_train_vit
Loaded X_test_tfidf
Loaded X_test_bert
Loaded X_test_cannyhog
Loaded X_test_resnet
Loaded X_test_vit
Loaded y_train
Loaded y_test


## Train-Validation Split

In [9]:
X, y = train_val_split(X, y)

## Scale Regression Targets

In [10]:
y, y_scaler = scale(y)

In [11]:
y = {subset: torch.tensor(_y, dtype=torch.float32, device=device) for subset, _y in y.items()}

## Dimensions

In [12]:
X_dim, y_dim = get_dimensions(X, y)

X Dimensions: {'tfidf': 300, 'bert': 768, 'cannyhog': 2917, 'resnet': 2048, 'vit': 768}
y Dimensions: 2


## Torch Datasets and Dataloaders

In [13]:
datasets = create_pottery_datasets(X, y)
loaders = create_pottery_dataloaders(datasets, batch_size=64)

## Tune Model



In [14]:
param_grid = {
    # Architecture Params
    "hidden_size": [128, 256, 512, 1024],
    "activation": [nn.ReLU, nn.GELU],
    "dropout": [0, 0.1, 0.3, 0.5],
    "blocks": [1, 2, 3, 5, 10],

    # Training Params
    "lr": [1e-2, 1e-3, 1e-4, 1e-5],
    "weight_decay": [1e-5, 1e-6, 1e-7],
}

In [15]:
tune(param_grid, X_dim["tfidf"], y_dim, device)


🔎 Running with params: {'activation': <class 'torch.nn.modules.activation.ReLU'>, 'blocks': 1, 'dropout': 0, 'hidden_size': 128, 'lr': 0.01, 'weight_decay': 1e-05}

🔎 Running with params: {'activation': <class 'torch.nn.modules.activation.ReLU'>, 'blocks': 1, 'dropout': 0, 'hidden_size': 128, 'lr': 0.01, 'weight_decay': 1e-06}

🔎 Running with params: {'activation': <class 'torch.nn.modules.activation.ReLU'>, 'blocks': 1, 'dropout': 0, 'hidden_size': 128, 'lr': 0.01, 'weight_decay': 1e-07}

🔎 Running with params: {'activation': <class 'torch.nn.modules.activation.ReLU'>, 'blocks': 1, 'dropout': 0, 'hidden_size': 128, 'lr': 0.001, 'weight_decay': 1e-05}

🔎 Running with params: {'activation': <class 'torch.nn.modules.activation.ReLU'>, 'blocks': 1, 'dropout': 0, 'hidden_size': 128, 'lr': 0.001, 'weight_decay': 1e-06}

🔎 Running with params: {'activation': <class 'torch.nn.modules.activation.ReLU'>, 'blocks': 1, 'dropout': 0, 'hidden_size': 128, 'lr': 0.001, 'weight_decay': 1e-07}

🔎 Runn